<h1><center>Function calc_mu_hmi.py</center>
    
<h3>Parameters used in calc_mu_hmi:</h3>
<ul>
	<li>thresh: the image converted to gray scale</li>
</ul>

In [1]:
from library import *

Using TensorFlow backend.


In [ ]:
try:    
    connectivity=8
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity)
    sizes = stats[1:, -1]
    nb_components = nb_components - 1
    min_size = 100000 
    imagemEmComponentes = np.zeros((output.shape))
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            imagemEmComponentes[output == i + 1] = 1

    labels = label(imagemEmComponentes)

    props = regionprops(labels)

    center_x = props[0]['Centroid'][0]
    center_y = props[0]['Centroid'][1]

    EquivDiameter = props[0]['EquivDiameter']

    mradius = EquivDiameter/2 

    #jx, jy = np.meshgrid(range(1,output.shape[0]+1), range(1,output.shape[1]+1)) #original
    jx, jy = np.meshgrid(range(output.shape[0]), range(output.shape[1]))

    jr = np.sqrt(np.power(jx-center_x,2) + np.power(jy-center_y,2))


    a = 1-np.power(jr/mradius,2)
    a = a.astype('complex')

    #mu = np.real(np.sqrt(a))

    mu = octave.real(octave.sqrt(a))

    ii0 = np.array([1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.05])
    ii1 = np.linspace(1,len(ii0),len(ii0)) #original
    #ii1 = range(1,12)

    ''' As linhas de 86 a 94 são a opção no caso de não utilizar o interpolador interp1.m
    mu_rings = np.empty_like(mu)
    mu_rings[:] = np.nan

    limites = [1, 0.95, 0.85, 0.75, 0.65, 0.55, 0.45, 0.35, 0.25, 0.15, 0.075]

    for i in range(len(limites)-1):
        mu_rings[np.where((mu>limites[i+1]) & (mu<=limites[i]))] = ii1[i]

    mu_rings[np.where((mu>=0) & (mu<limites[10]))] = ii1[10]

    '''

    mu_rings = octave.interp1(ii0, ii1, mu, 'nearest')

    #mu_rings = interp1d(ii0, ii1, kind = 'nearest', fill_value = 'extrapolate')(mu)

    return mu, mu_rings

except:
    print('Erro na função calc_mu_rings')
    return (None, None)
